In [1]:
import json
import pickle
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
from collections import defaultdict

In [ ]:
# result_file_path = "../result/file_level_base/04-11_21-54_zero_prompt/final_results_phi4:14b.json"
# result_file_path = "../result/file_level_base/04-11_21-54_cot_prompt/final_results_phi4:14b.json"
# result_file_path = "../result/file_level_base/04-11_21-54_fs_prompt/final_results_phi4:14b.json"

# result_file_path = "../result/FT_file_result/04-18_08-37_zero_prompt/final_results_Phi4_FT_3ep_file.json"
# result_file_path = "../result/FT_file_result/04-18_08-37_cot_prompt/final_results_Phi4_FT_3ep_file.json"
# result_file_path = "../result/FT_file_result/04-18_08-37_fs_prompt/final_results_Phi4_FT_3ep_file.json"

# result_file_path = "../result/FT_file_explain/04-22_11-13_zero_prompt/final_results_FT_file_ep3_explain.json"
# result_file_path = "../result/FT_file_explain/04-22_11-13_cot_prompt/final_results_FT_file_ep3_explain.json"
# result_file_path = "../result/FT_file_explain/04-22_11-13_fs_prompt/final_results_FT_file_ep3_explain.json"

# result_file_path = "../result/func_level_base/04-11_17-55_zero_prompt/final_results_phi4:14b.json"
result_file_path = "../result/func_level_base/04-10_09-29_cot_prompt/final_results_phi4:14b.json"
# result_file_path = "../result/func_level_base/04-10_09-29_fs_prompt/final_results_phi4:14b.json"

# result_file_path = "../result/FT_func_result_ver2/04-19_22-49_zero_prompt/final_results_FT_func_ep3_v2.json"
# result_file_path = "../result/FT_func_result_ver2/04-19_22-49_cot_prompt/final_results_FT_func_ep3_v2.json"
# result_file_path = "../result/FT_func_result_ver2/04-19_22-49_fs_prompt/final_results_FT_func_ep3_v2.json"

# result_file_path = "../result/FT_func_explain/04-21_11-56_zero_prompt/final_results_FT_func_ep3_explain.json"
# result_file_path = "../result/FT_func_explain/04-21_11-56_cot_prompt/final_results_FT_func_ep3_explain.json"
# result_file_path = "../result/FT_func_explain/04-21_11-56_fs_prompt/final_results_FT_func_ep3_explain.json"

# result_file_path = "../result/FT_file_explain_simple_v2/05-09_00-12_zero_prompt/final_results_FT_file_explain_simple.json"
# result_file_path = "../result/FT_file_explain_simple_v2/05-09_00-12_cot_prompt/final_results_FT_file_explain_simple.json"
# result_file_path = "../result/FT_file_explain_simple_v2/05-09_00-12_fs_prompt/final_results_FT_file_explain_simple.json"

# result_file_path = "../result/FT_func_explain_simple_v2/05-09_15-43_zero_prompt/final_results_FT_func_explain_simple.json"
# result_file_path = "../result/FT_func_explain_simple_v2/05-09_15-43_cot_prompt/final_results_FT_func_explain_simple.json"
# result_file_path = "../result/FT_func_explain_simple_v2/05-09_15-43_fs_prompt/final_results_FT_func_explain_simple.json"

# Load the JSON file
with open(result_file_path, 'r') as file:
    results = json.load(file)

print(len(results))

411


# Result Analysis

In [3]:
from random import randint 
print(json.dumps(results[randint(0, len(results))], indent=4))

{
    "index": 46,
    "file_change_id": "73229817965895",
    "lang": "JavaScript",
    "vuln_type": "CWE-79",
    "result": {
        "reasoning": "The provided code snippet appears to be part of a JavaScript library, likely related to graph visualization (possibly mxGraph). The code involves iterating over elements and applying certain operations based on their type. Specifically, it checks if an element is a table cell or row and performs actions accordingly. Additionally, it modifies the behavior of `createParentHighlightShape` for vertex and edge handlers.\n\nThe snippet does not involve any direct handling of user input that could lead to vulnerabilities such as Cross-Site Scripting (CWE-79) or SQL Injection (CWE-89). There is no evidence of file uploads, which would be relevant for CWE-434. Similarly, there are no indications of cross-site request forgery mechanisms, which would relate to CWE-352.\n\nThe operations performed in the code are internal manipulations within a graph

In [4]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def calculate__metrics(true_label, pred_label):
  """
  Calculate weighted and macro classification metrics for imbalanced multi-class data.
  
  Args:
    true_label (array-like): Ground truth labels.
    pred_label (array-like): Predicted labels.
    
  Returns:
    dict: Metrics including Accuracy, Precision, F1-score, FNR, and FPR (both weighted and macro).
  """
  cwe_classes = np.unique(true_label)
  cm = confusion_matrix(true_label, pred_label, labels=cwe_classes)
  support = cm.sum(axis=1)  # Number of samples per class

  # True positives per class
  tp = np.diag(cm)
  # False positives: predicted as class i but actually another class
  fp = cm.sum(axis=0) - tp
  # False negatives: actually class i but predicted as another
  fn = cm.sum(axis=1) - tp
  # True negatives: all others
  tn = cm.sum() - (tp + fp + fn)

  # Per-class FNR and FPR
  fnr = fn / (fn + tp + 1e-12)  # Avoid division by zero
  fpr = fp / (fp + tn + 1e-12)

  # Weighted (support-based) averages
  weighted_fnr = np.average(fnr, weights=support)
  weighted_fpr = np.average(fpr, weights=support)

  # Macro (unweighted) averages
  macro_fnr = np.mean(fnr)
  macro_fpr = np.mean(fpr)

  # Metrics from sklearn
  acc = accuracy_score(true_label, pred_label)
  precision = precision_score(true_label, pred_label, average='weighted', zero_division=0)
  f1 = f1_score(true_label, pred_label, average='weighted', zero_division=0)
  macro_f1 = f1_score(true_label, pred_label, average='macro', zero_division=0)

  return {
    "Weighted Accuracy": acc,
    "Weighted Precision": precision,
    "Weighted F1-score": f1,
    "Weighted FNR": weighted_fnr,
    "Weighted FPR": weighted_fpr,
    "Macro F1-score": macro_f1,
    "Macro FNR": macro_fnr,
    "Macro FPR": macro_fpr
  }

In [5]:
chosen_cwes = ['79', '89', '434', '352']
langs = ['PHP', 'JavaScript', 'Java', 'TypeScript', 'Ruby', 'Python']

def judge_cwe(cwe):
    for chosen_cwe in chosen_cwes:
        if chosen_cwe in cwe:
            return 'CWE-'+ chosen_cwe
    return 'None'

for lang in langs:
  true_label = []
  pred_label = []
  for result in results:
    if result['lang'] == lang:
      true = result['vuln_type']

      if true == 'non_vuln': true = 'None'

      if(result['result'].get('cwe_id') != None):
        pred = judge_cwe(result['result']['cwe_id'])
        true_label.append(true)
        pred_label.append(pred)

    
  metrics = calculate__metrics(true_label, pred_label)
  print(f"Language: {lang}")
  for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


Language: PHP
Weighted Accuracy: 0.3986
Weighted Precision: 0.4136
Weighted F1-score: 0.3280
Weighted FNR: 0.6014
Weighted FPR: 0.3104
Macro F1-score: 0.2337
Macro FNR: 0.7474
Macro FPR: 0.1824
Language: JavaScript
Weighted Accuracy: 0.5188
Weighted Precision: 0.6866
Weighted F1-score: 0.3768
Weighted FNR: 0.4812
Weighted FPR: 0.4879
Macro F1-score: 0.1498
Macro FNR: 0.7958
Macro FPR: 0.1938
Language: Java
Weighted Accuracy: 0.7429
Weighted Precision: 0.6357
Weighted F1-score: 0.6739
Weighted FNR: 0.2571
Weighted FPR: 0.5741
Macro F1-score: 0.4772
Macro FNR: 0.5517
Macro FPR: 0.2078
Language: TypeScript
Weighted Accuracy: 0.6429
Weighted Precision: 0.5659
Weighted F1-score: 0.5425
Weighted FNR: 0.3571
Weighted FPR: 0.4762
Macro F1-score: 0.3155
Macro FNR: 0.6667
Macro FPR: 0.2083
Language: Ruby
Weighted Accuracy: 0.5417
Weighted Precision: 0.4167
Weighted F1-score: 0.4702
Weighted FNR: 0.4583
Weighted FPR: 0.5806
Macro F1-score: 0.2339
Macro FNR: 0.7375
Macro FPR: 0.2597
Language: Pyth